<a href="https://colab.research.google.com/github/Nimalnix/AIML/blob/main/naive_bayes_1_titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Naive Bayes Tutorial Part 1: Predicting survival from titanic crash**

In [1]:
import pandas as pd

In [2]:
from google.colab import files

uploaded = files.upload()

# Read the uploaded file
import io
df = pd.read_csv(io.BytesIO(uploaded['titanic.csv']))
print(df)

Saving titanic.csv to titanic.csv
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                 

In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female

In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [14]:
model.fit(X_train,y_train)

GaussianNB()

In [16]:
model.score(X_test,y_test)

0.7574626865671642

In [17]:
X_test[0:10]

,Pclass,Age,Fare,female
650,3,29.699118,7.8958,0
457,1,29.699118,51.8625,1
4,3,35.000000,8.0500,0
836,3,21.000000,8.6625,0
632,1,32.000000,30.5000,0
837,3,29.699118,8.0500,0
703,3,25.000000,7.7417,0
311,1,18.000000,262.3750,1
651,2,18.000000,23.0000,1
275,1,63.000000,77.9583,1


In [18]:
y_test[0:10]

650    0
457    1
4      0
836    0
632    1
837    0
703    0
311    1
651    1
275    1
Name: Survived, dtype: int64

In [19]:
model.predict(X_test[0:10])

array([0, 1, 0, 0, 0, 0, 0, 1, 1, 1])

In [21]:
model.predict_proba(X_test[:10])

array([[9.65665802e-01, 3.43341982e-02],
       [3.85224558e-02, 9.61477544e-01],
       [9.66829461e-01, 3.31705391e-02],
       [9.60281090e-01, 3.97189098e-02],
       [7.90815044e-01, 2.09184956e-01],
       [9.65682354e-01, 3.43176460e-02],
       [9.63313720e-01, 3.66862796e-02],
       [8.50979561e-10, 9.99999999e-01],
       [1.33766735e-01, 8.66233265e-01],
       [1.09537884e-02, 9.89046212e-01]])

Calculate the score using cross validation

In [22]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.776     , 0.8       , 0.8       , 0.75      , 0.80645161])